# Evaluation of NQPV

NQPV process ".nqpv" files in the local folder. 
For convenience we prepare the method to verify a string of program program directly.

In [ ]:
import nqpv

def verify_prog(content : str):
    fp = open("example.nqpv","w")
    fp.write(content)
    fp.close()

    nqpv.entrance.verify("example.nqpv")

## NQPV - Introduction and Demonstration

This section introduces the NQPV system, and a detailed explanation may be found in the "README.md" file.

### Hello World Example

This is a hello-world example.

In [ ]:
code = '''
def pf := proof [q] :
    { P0[q] };
    q *= X;
    { P1[q] }
end

show pf end
'''
verify_prog(code)

The above verification corresponds to this correctness formula:
$$
\{\ket{0}_{q}\bra{0}\}\ q\ *= X\ \{\ket{1}_{q}\bra{1}\},
$$
which says that the $X$ gate flips $\ket{0}$ state to $\ket{1}$ state.

### Variable system

A simple variable system is implemented. Variables are saved in scopes, and some commonly used operators are prepared in the global scope. The following command displays the global scope.

In [ ]:
code = '''
    show global end
'''
verify_prog(code)

We can display the variables. For example, the measurement operators for $\ket{0}, \ket{1}$ basis and $\ket{+}, \ket{-}$ basis.

In [ ]:
code = '''
    show M10 end
    show global.Mpm end
'''
verify_prog(code)

### Verifications Ability

#### Detect syntax error

Besides the syntax check while parsing, NQPV also check whether the given quantum program is valid. For example, the number of qubits should match the quantum variable list, and there should not be identical variables in a variable list.

In [ ]:
code = '''
    def pf := proof [q] :
        { I[q] };
        q *= CX;
        { I[q]}
    end

    def pf := proof [q1 q2] :
        { I[q1] };
        [q1 q1] *= CX;
        { I[q1] }
    end
'''
verify_prog(code)

#### Detect invalid correctness formula

The following correctness proof does not hold, and is not accepted by NQPV.

In [ ]:
code = '''
    def pf := proof [q] :
        { P0[q] };
        q *= X;
        { P0[q]}
    end
'''
verify_prog(code)

#### Human support and loop invariant

To prove the correctness of loops, a loop invariant as hint is required in the program. A weak loop invariant may result in the rejection of a theoretically true correctness formula.

In [ ]:
code = '''
    def pf := proof [q] :
        { I[q] };
        { inv : Zero[q] };
        while M10[q] do
            q *= H
        end;
        { P0[q] }
    end
'''
verify_prog(code)

With a propriate invariant, NQPV accepts the formula.

In [ ]:
code = '''
    def pf := proof [q] :
        { I[q] };
        { inv : I[q] };
        while M10[q] do
            q *= H
        end;
        { P0[q] }
    end
    show pf end
'''
verify_prog(code)

### Soundness and Precision

Since NQPV is based on numerical methods, it is influnced by the precision settings. According to our experiences, NQPV is still a sound tool. Correctness proofs accepted by NQPV are also provable theoretically.

If the equality judgement precision (EPS) is higher than the precision of the SDP solver (SDP_PRECISION), then NQPV may reject a correctness proof which actually holds.

In [ ]:
code = '''
    setting EPS := 1e-8 end
    setting SDP_PRECISION := 1e-5 end
    def pf := proof [q] :
        { P0[q] P1[q] };
        q *= X;
        { P1[q] P0[q] }
    end
'''
verify_prog(code)

By an attemp of higher SDP solve precision, we are still able to prove this correctness formula.

In [ ]:
code = '''
    setting EPS := 1e-8 end
    setting SDP_PRECISION := 1e-10 end
    def pf := proof [q] :
        { P0[q] P1[q] };
        q *= X;
        { P1[q] P0[q] }
    end
    show pf end
'''
verify_prog(code)

## Verifying Article Results

### Error Correction Code

Because NQPV is a numerical tool, it cannot handle the statement that error correction code works for any (parametrized) initial states. So here we check a random initial state.

In [ ]:
import nqpv
import numpy as np

# create a Hermitian projection operator on a random ket
theta = np.random.rand() * np.pi
phi = np.random.rand() * np.pi * 2

ket = np.array([np.cos(theta), np.sin(theta)*np.exp(phi*1j)])

Hrand = np.outer(ket, np.conj(ket))

np.save("Hrand", Hrand)

# the program
code = '''
def Hrand := load "Hrand.npy" end

def pf := proof[q] :
    { Hrand[q] };

    [q1 q2] :=0;
    [q q1] *= CX;
    [q q2] *= CX;
    (skip # q *= X # q1 *= X # q2 *= X);
    [q q1] *= CX;
    [q q2] *= CX;
    [q1 q2 q] *= CCX;

    { Hrand[q] }
end

show pf end
'''

# verify
verify_prog(code)

Examing the proof outline, we will find that the generated verification condition is named "VAR13", and we can compare it with the precondition we gave.

In [ ]:
code = '''
def Hrand := load "Hrand.npy" end

def pf := proof[q] :
    { Hrand[q] };

    [q1 q2] :=0;
    [q q1] *= CX;
    [q q2] *= CX;
    (skip # q *= X # q1 *= X # q2 *= X);
    [q q1] *= CX;
    [q q2] *= CX;
    [q1 q2 q] *= CCX;

    { Hrand[q] }
end

show Hrand end
show VAR13 end
'''

# verify
verify_prog(code)

They are actualy identical with respect to a cylinder extension. In other words,
$$
\mathrm{Hrand} = \mathrm{VAR13}\otimes I \otimes I.
$$

### Quantum Walk

In [ ]:
import nqpv
import numpy as np

# create the required operators
W1 = np.array([[1., 1., 0., -1.],
                [1., -1., 1., 0.],
                [0., 1., 1., 1.],
                [1., 0., -1., 1.]]) / np.sqrt(3)
W2 = np.array([[1., 1., 0., 1.],
                [-1., 1., -1., 0.],
                [0., 1., 1., -1.],
                [1., 0., -1., -1.]]) / np.sqrt(3)
np.save("W1", W1.reshape((2,2,2,2)))
np.save("W2", W2.reshape((2,2,2,2)))

P0 = np.array([[0., 0., 0., 0.],
                    [0., 0., 0., 0.],
                    [0., 0., 1., 0.],
                    [0., 0., 0., 0.]])
P1 = np.array([[1., 0., 0., 0.],
                    [0., 1., 0., 0.],
                    [0., 0., 0., 0.],
                    [0., 0., 0., 1.]])
                    
MQWalk = np.stack((P1,P0), axis = 0)
np.save("MQWalk", MQWalk.reshape((2,2,2,2,2)))

# the invariant N
invN = np.array([[1., 0., 0., 0.],
                [0., 0.5, 0., 0.5],
                [0., 0., 0., 0.],
                [0., 0.5, 0., 0.5]])
np.save("invN", invN.reshape((2,2,2,2)))

# the program of quantum walk
code = '''
def invN := load "invN.npy" end
def MQWalk := load "MQWalk.npy" end
def W1 := load "W1.npy" end
def W2 := load "W2.npy" end

def pf := proof[q1 q2] :
    { I[q1] };

    [q1 q2] :=0;

    {inv: invN[q1 q2]};
    while MQWalk[q1 q2] do
        (
            [q1 q2] *= W1; [q1 q2] *= W2
        #
            [q1 q2] *= W2; [q1 q2] *= W1
        )
    end;

    { Zero[q1] }

end

show pf end
'''
# verify
verify_prog(code)

Check the verification condition.

In [ ]:
# the program of quantum walk
code = '''
def invN := load "invN.npy" end
def MQWalk := load "MQWalk.npy" end
def W1 := load "W1.npy" end
def W2 := load "W2.npy" end

def pf := proof[q1 q2] :
    { I[q1] };

    [q1 q2] :=0;

    {inv: invN[q1 q2]};
    while MQWalk[q1 q2] do
        (
            [q1 q2] *= W1; [q1 q2] *= W2
        #
            [q1 q2] *= W2; [q1 q2] *= W1
        )
    end;

    { Zero[q1] }

end

show I end
show VAR2 end
'''
# verify
verify_prog(code)

### Deutsch algorithm

In [ ]:
import nqpv
import numpy as np

# create the required operators
Hpost = np.array([[1., 0., 0., 0.],
                    [0., 0., 0., 0.],
                    [0., 0., 0., 0.],
                    [0., 0., 0., 1.]])
np.save("./Hpost", Hpost.reshape((2,2,2,2)))


# the program of Deutsch algorithm
code = '''
def Hpost := load "Hpost.npy" end

def pf := proof[q q1] :
    { I[q] };
    [q1 q2] :=0;
    q1 *= H;
    q2 *= X;
    q2 *= H;
    if M10[q] then
        ( 
            [q1 q2] *= CX
        #
            q1 *= X;
            [q1 q2] *= CX;
            q1 *= X
        )
    else
        (
            skip
        #
            q2 *= X
        )
    end;
    q1 *= H;
    if M01[q1] then
        skip
    else
        skip
    end;
    { Hpost[q q1] }
end

show pf end
'''

# verify
verify_prog(code)

## Performance Test: n-qubit Grover algorithm

It is hard to fairly compare the "performance" of different tools. Even though, here we take the Grover algorithm as an example to test the performance of NQPV on programs with large qubit number.

In [ ]:
from examples import GroverN

This is the program for 2-qubit Grover algorithm. Again, here we assume a particular solution of the oracle, and the correctness formula states that the algorithm successfully finds the solution with certaion probability.

In [ ]:
print(GroverN.generate_grover_prog(2))

The following performance test will cost several minutes.

In [ ]:
n_max = 13
GroverN.grover_performance_test(n_max)